In [1]:
import numpy as np
import pandas as pd


# Load the data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
# sample_submission = pd.read_csv("sample_submission.csv")
# sample_submission_y = sample_submission.drop(columns=["UID"])

# Extract features and target
X_train = train_data.iloc[500:, 1:-1]
y_train = train_data.iloc[500:, -1]
X_val = train_data.iloc[:500, 1:-1]
y_val = train_data.iloc[:500, -1]
X_test = test_data.iloc[:, 1:]

# Categorical columns
categorical_cols = ['col_0', 'col_1', 'col_2', 'col_3', 'col_5', 'col_6']

# # Fill NaN values in categorical columns with 'Unknown'
# def fill_na(data, categorical_cols):
#     for col in categorical_cols:
#         data[col].fillna('Unknown', inplace=True)
#     return data

# Fill NaN values in categorical columns with the most frequent value in the respective columns
def fill_na(data, categorical_cols):
    for col in categorical_cols:
        most_frequent_value = data[col].mode().iloc[0]  # Get the mode (most frequent value)
        data[col].fillna(most_frequent_value, inplace=True)
    return data

X_train = fill_na(X_train, categorical_cols)
X_val = fill_na(X_val, categorical_cols)
X_test = fill_na(X_test, categorical_cols)

# label encoding
def label_encoding(data, categorical_cols):
    label_encoders = {}
    for col in categorical_cols:
        unique_categories = data[col].unique()
        encoding_map = {category: index + 1 for index, category in enumerate(unique_categories)}
        data[col] = data[col].map(encoding_map)
        label_encoders[col] = encoding_map
    return data

X_train = label_encoding(X_train, categorical_cols)
X_val = label_encoding(X_val, categorical_cols)
X_test = label_encoding(X_test, categorical_cols)

# normalize
def normalize(df):
    return (df - df.min()) / (df.max() - df.min())

X_train = normalize(X_train)
X_val = normalize(X_val)
X_test = normalize(X_test)

# convert df into numpy
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
X_test = X_test.to_numpy()
y_train=y_train.reshape(-1,1)
# sample_submission_y = sample_submission_y.to_numpy()

In [2]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [3]:
class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

In [4]:
print(X_train.shape)
print(y_train.shape)

(2128, 7)
(2128, 1)


In [5]:
model=DecisionTreeRegressor(min_samples_split=3,max_depth=3)
model.fit(X_train,y_train)
model.print_tree()

X_5 <= 0.0 ? 897575474.0409102
 left:X_3 <= 0.03076923076923077 ? 106344703.80572653
  left:X_2 <= 0.024390243902439025 ? 86525327.96161318
    left:X_2 <= 0.012195121951219513 ? 240663238.01804113
        left:155571.06397306398
        right:119528.39655172414
    right:X_6 <= 0.08333333333333333 ? 85101317.84991264
        left:162813.2343387471
        right:187585.7906976744
  right:X_2 <= 0.7682926829268293 ? 393145470.7201092
    left:X_2 <= 0.012195121951219513 ? 154786330.73662472
        left:43738.375
        right:69860.2
    right:160000.0
 right:X_3 <= 0.13846153846153847 ? 178401312.9277792
  left:X_5 <= 0.04918032786885246 ? 397802892.4967532
    left:X_6 <= 0.3333333333333333 ? 190605226.2003894
        left:70100.80291970803
        right:165440.66666666666
    right:X_0 <= 0.06666666666666667 ? 390079467.46438074
        left:124165.6875
        right:77155.94736842105
  right:X_6 <= 0.9166666666666666 ? 218338850.90124178
    left:X_3 <= 0.16923076923076924 ? 174233

In [6]:
def calculate_rmse(y_true, y_pred):
    squared_errors = (y_true - y_pred) ** 2
    mean_squared_error = np.mean(squared_errors)
    rmse = np.sqrt(mean_squared_error)
    return rmse

In [7]:
#calculate rmse on the val set
val_pred=model.predict(X_val)
val_rmse=calculate_rmse(val_pred,y_val)
print(val_rmse)


56840.51234364615


In [8]:
# prediction on the test set
test_pred=model.predict(X_test)
result_df = pd.DataFrame({'UID': test_data['UID'], 'y': test_pred})
# Save predictions
result_df.to_csv('DecisionTree.csv', index=False)

# all code in a single block

In [10]:
import numpy as np
import pandas as pd

# Load the data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
# sample_submission = pd.read_csv("sample_submission.csv")
# sample_submission_y = sample_submission.drop(columns=["UID"])

# Extract features and target
X_train = train_data.iloc[500:, 1:-1]
y_train = train_data.iloc[500:, -1]
X_val = train_data.iloc[:500, 1:-1]
y_val = train_data.iloc[:500, -1]
X_test = test_data.iloc[:, 1:]

# Categorical columns
categorical_cols = ['col_0', 'col_1', 'col_2', 'col_3', 'col_5', 'col_6']

# Fill NaN values in categorical columns with 'Unknown'
def fill_na(data, categorical_cols):
    for col in categorical_cols:
        data[col].fillna('Unknown', inplace=True)
    return data

X_train = fill_na(X_train, categorical_cols)
X_val = fill_na(X_val, categorical_cols)
X_test = fill_na(X_test, categorical_cols)

# label encoding
def label_encoding(data, categorical_cols):
    label_encoders = {}
    for col in categorical_cols:
        unique_categories = data[col].unique()
        encoding_map = {category: index + 1 for index, category in enumerate(unique_categories)}
        data[col] = data[col].map(encoding_map)
        label_encoders[col] = encoding_map
    return data

X_train = label_encoding(X_train, categorical_cols)
X_val = label_encoding(X_val, categorical_cols)
X_test = label_encoding(X_test, categorical_cols)

# normalize
def normalize(df):
    return (df - df.min()) / (df.max() - df.min())

X_train = normalize(X_train)
X_val = normalize(X_val)
X_test = normalize(X_test)

# convert df into numpy
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
X_test = X_test.to_numpy()
y_train=y_train.reshape(-1,1)
# sample_submission_y = sample_submission_y.to_numpy()


class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value


class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

model=DecisionTreeRegressor(min_samples_split=3,max_depth=3)
model.fit(X_train,y_train)
model.print_tree()

X_5 <= 0.0 ? 897575474.0409102
 left:X_3 <= 0.03076923076923077 ? 106344703.80572653
  left:X_2 <= 0.024390243902439025 ? 86525327.96161318
    left:X_2 <= 0.012195121951219513 ? 240663238.01804113
        left:155571.06397306398
        right:119528.39655172414
    right:X_6 <= 0.15384615384615385 ? 85101317.84991264
        left:162813.2343387471
        right:187585.7906976744
  right:X_2 <= 0.7682926829268293 ? 393145470.7201092
    left:X_2 <= 0.012195121951219513 ? 154786330.73662472
        left:43738.375
        right:69860.2
    right:160000.0
 right:X_3 <= 0.13846153846153847 ? 178401312.9277792
  left:X_5 <= 0.04918032786885246 ? 397802892.4967532
    left:X_6 <= 0.38461538461538464 ? 190605226.2003894
        left:70100.80291970803
        right:165440.66666666666
    right:X_5 <= 0.16393442622950818 ? 328421091.68214035
        left:117482.27848101266
        right:32866.75
  right:X_6 <= 0.9230769230769231 ? 218338850.90124178
    left:X_3 <= 0.16923076923076924 ? 1742337

# prunnig

In [11]:
import numpy as np
import pandas as pd

# Load the data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Extract features and target
X_train = train_data.iloc[500:, 1:-1]
y_train = train_data.iloc[500:, -1]
X_val = train_data.iloc[:500, 1:-1]
y_val = train_data.iloc[:500, -1]
X_test = test_data.iloc[:, 1:]

# Categorical columns
categorical_cols = ['col_0', 'col_1', 'col_2', 'col_3', 'col_5', 'col_6']

# Fill NaN values in categorical columns with 'Unknown'
def fill_na(data, categorical_cols):
    for col in categorical_cols:
        data[col].fillna('Unknown', inplace=True)
    return data

X_train = fill_na(X_train, categorical_cols)
X_val = fill_na(X_val, categorical_cols)
X_test = fill_na(X_test, categorical_cols)

# label encoding
def label_encoding(data, categorical_cols):
    label_encoders = {}
    for col in categorical_cols:
        unique_categories = data[col].unique()
        encoding_map = {category: index + 1 for index, category in enumerate(unique_categories)}
        data[col] = data[col].map(encoding_map)
        label_encoders[col] = encoding_map
    return data

X_train = label_encoding(X_train, categorical_cols)
X_val = label_encoding(X_val, categorical_cols)
X_test = label_encoding(X_test, categorical_cols)

# normalize
def normalize(df):
    return (df - df.min()) / (df.max() - df.min())

X_train = normalize(X_train)
X_val = normalize(X_val)
X_test = normalize(X_test)

# convert df into numpy
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.reshape(-1, 1)

class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor '''
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        # for leaf node
        self.value = value

class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2, min_samples_leaf=1):
        ''' constructor '''
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf  # New parameter for minimum samples in a leaf

    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''
        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        
        if (
            num_samples >= self.min_samples_split and
            curr_depth <= self.max_depth
        ):
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            
            # check if information gain is positive and apply pruning
            if (
                best_split.get("var_red", 0) > 0 and
                num_samples >= self.min_samples_leaf  # Pruning condition
            ):
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(
                    best_split["feature_index"], best_split["threshold"],
                    left_subtree, right_subtree, best_split["var_red"]
                )
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red > max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_" + str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        if tree.value != None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

# Create the model with pruning
model = DecisionTreeRegressor(min_samples_split=3, max_depth=3, min_samples_leaf=5)
model.fit(X_train, y_train)
model.print_tree()


X_5 <= 0.0 ? 897575474.0409102
 left:X_3 <= 0.03076923076923077 ? 106344703.80572653
  left:X_2 <= 0.024390243902439025 ? 86525327.96161318
    left:X_2 <= 0.012195121951219513 ? 240663238.01804113
        left:155571.06397306398
        right:119528.39655172414
    right:X_6 <= 0.15384615384615385 ? 85101317.84991264
        left:162813.2343387471
        right:187585.7906976744
  right:X_2 <= 0.7682926829268293 ? 393145470.7201092
    left:X_2 <= 0.012195121951219513 ? 154786330.73662472
        left:43738.375
        right:69860.2
    right:160000.0
 right:X_3 <= 0.13846153846153847 ? 178401312.9277792
  left:X_5 <= 0.04918032786885246 ? 397802892.4967532
    left:X_6 <= 0.38461538461538464 ? 190605226.2003894
        left:70100.80291970803
        right:165440.66666666666
    right:X_5 <= 0.16393442622950818 ? 328421091.68214035
        left:117482.27848101266
        right:32866.75
  right:X_6 <= 0.9230769230769231 ? 218338850.90124178
    left:X_3 <= 0.16923076923076924 ? 1742337

In [12]:
sample_submission_data=pd.read_csv("sample_submission.csv")
sample_y=sample_submission_data['y']
sample_y=sample_y.to_numpy()

In [13]:

#prediction and rmse on train set
train_predict=model.predict(X_train)
train_rmse=calculate_rmse(train_predict,y_train)

#prediction and rmse on train set
val_predict=model.predict(X_val)
val_rmse=calculate_rmse(val_predict,y_val)

#prediction and rmse on test set
test_predict=model.predict(X_test)
test_rmse=calculate_rmse(test_predict,sample_y)
print("train rmse",train_rmse)
print("val rmse",val_rmse)
print("test rmse",test_rmse)

train rmse 72829.53996785905
val rmse 57990.18174766935
test rmse 149514.35776517144
